In [ ]:
from google.colab import files
import io

realCsv = files.upload()

In [ ]:
import pandas as pd

dfFake = pd.read_csv('./drive/My Drive/CS3244/Fake.csv')
# io.StringIO(uploaded['Fake.csv'].decode("utf-8"))

fakeLabel = [0 for x in range(dfFake.shape[0])]
dfFake['true_label'] = fakeLabel

dfFake['subject_id'] = dfFake['subject'].factorize()[0]
category_id_df = dfFake[['subject', 'subject_id']].drop_duplicates().sort_values('subject_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['subject_id', 'subject']].values)

In [ ]:
dfTrue = pd.read_csv('./drive/My Drive/CS3244/True.csv')

trueLabel = [1 for x in range(dfTrue.shape[0])]
dfTrue['true_label'] = trueLabel

dfTrue['subject_id'] = dfTrue['subject'].factorize()[0]

category_id_df = dfTrue[['subject', 'subject_id']].drop_duplicates().sort_values('subject_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['subject_id', 'subject']].values)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split

li = []
li.append(dfFake)
li.append(dfTrue)
df = pd.concat(li, ignore_index=True)

# Split the data into 80% for training and 20% for testing.
# (random_state): we use a fixed random seed so we get the same results every time.
X_train, X_test, y_train, y_test = train_test_split(df, df.true_label, test_size=0.2, random_state=0)

# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=7, norm='l2', encoding='utf-8', ngram_range=(1, 1), stop_words='english')
# features = tfidf.fit_transform(df.text).toarray()
# labels = df.subject_id
# features.shape

vectorizer = HashingVectorizer(n_features=2**15, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words=None, alternate_sign=False)
train_features = vectorizer.fit_transform(X_train.text)
test_features = vectorizer.fit_transform(X_test.text)

print(train_features.shape)
print(test_features.shape)

(35918, 32768)
(8980, 32768)


In [ ]:
import gc 

# df.sample(5, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(train_features, y_train)

print("Naïve Bayes accuracy for train set: %f" % nb_clf.score(train_features, y_train))
print("Naïve Bayes accuracy for test set: %f" % nb_clf.score(test_features, y_test))

Naïve Bayes accuracy for train set: 0.957431
Naïve Bayes accuracy for test set: 0.954232


**Multinomial Naive Bayes hyperparams**

L2 Norm + Stop_words='english'
1.   n_features=2**14, ngram_range=(1, 1), train: 0.943037, test: 0.939755
2.   n_features=2**14, ngram_range=(1, 2), train: 0.951389, test: 0.949777
3.   n_features=2**15, ngram_range=(1, 2), train: 0.955454, test: 0.952227
4.   n_features=2**15, ngram_range=(2, 2), train: 0.964113, test: 0.953229

5.   n_features=2**12, ngram_range=(1, 3), train: 0.942480, test: 0.941648
6.   n_features=2**14, ngram_range=(1, 3), train: 0.952809, test: 0.949443
6.   n_features=2**15, ngram_range=(1, 3), train: 0.956930, test: 0.952895

L2 Norm + Stop_words=None
1.   n_features=2**14, ngram_range=(1, 2), train: 0.951751, test: 0.953452
1.   n_features=2**15, ngram_range=(1, 2), train: 0.955120, test: 0.953563

L1 Norm + Stop_words='english'
1.   n_features=2**15, ngram_range=(1, 2), train: 0.889053, test: 0.888419





In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomforest_clf = RandomForestClassifier(n_estimators=400, max_depth=7, random_state=2)
randomforest_clf.fit(train_features, y_train)

print("RF accuracy for train set: %f" % randomforest_clf.score(train_features, y_train))
print("RF accuracy for test set: %f" % randomforest_clf.score(test_features, y_test))

RF accuracy for train set: 0.987110
RF accuracy for test set: 0.984855


**Random Forest hyperparams**

2^14 feature vectors
1.   n_estimators=100, max_depth=2, train set: 0.950999, test set: 0.948998
2.   n_estimators=100, max_depth=3, train set: 0.968512, test set: 0.962361
3.   n_estimators=200, max_depth=2, train set: 0.953032, test set: 0.948664

4.   n_estimators=200, max_depth=3, train set: 0.968094, test set: 0.963252

5.   n_estimators=100, max_depth=4, train set: 0.974191, test set: 0.970267
6.   n_estimators=100, max_depth=5, train set: 0.977226, test set: 0.973051
7.   **n_estimators=100, max_depth=6, train set: 0.980595, test set: 0.977171**
8.   **n_estimators=200, max_depth=6, train set: 0.982655, test set: 0.978731**

2^15 feature vectors
1.   n_estimators=100, max_depth=5, train set: 0.966340, test set: 0.961470
2.   n_estimators=200, max_depth=5, train set: 0.977282, test set: 0.975501
3.   **n_estimators=200, max_depth=6, train set: 0.981319, test set: 0.980290**
4.   **n_estimators=300, max_depth=6, train set: 0.986274, test set: 0.985078**
5.   **n_estimators=400, max_depth=6, train set: 0.985244, test set: 0.985078**
6.   **n_estimators=400, max_depth=7, train set: 0.987110, test set: 0.984855**
